In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize
import uproot as ur
import awkward as ak
import time as t
import copy
print("Awkward Array Version: "+str(ak.__version__))
print("Uproot Version: "+str(ur.__version__))

import energyflow as ef
from energyflow.archs import EFN
from energyflow.archs import PFN
# from energyflow.datasets import qg_jets
from energyflow.utils import data_split, to_categorical

from sklearn.metrics import roc_auc_score, roc_curve

Awkward Array Version: 1.7.0
Uproot Version: 4.1.9


In [4]:
import sys
sys.path.append('D:\Work\EPE\ML4pi')
from util import graph_util as gu

In [5]:
def find_sampling_events(_ak_cluster_cell_ID, _cellGeoID, _cellGeoSampling, _filter):
    ''' Inputs:
    _ak_cluster_cell_ID: awkward array with nested arrays of all the cell geometry IDs
    _cellGeoID: list of the cell IDs to find the index from, expects numpy format
    _cellGeoSampling: same as _cellGeoID but the sampling layer
    _filter: the numbered index of calorimetry layers
    Returns:
    an array of event indices with clusters containing barrel events
    '''

    # create unique list of sampling layers for every event
    _event_layers = []
    # list of event indices with one or more clusters in EMB1-3 or TileBar0-2
    _barrel_events = []
    
    for _evt_idx in range(len(_ak_cluster_cell_ID)):
        _cluster_cell_ID = ak_cluster_cell_ID["cluster_cell_ID"][_evt_idx]

        # create unique list of sampling layers for every cluster
        _event_layers = []

        for j in range(len(_cluster_cell_ID)):
            ''' loop through each cluster of the event '''

            for l in range(len(_cluster_cell_ID[j])):
                ''' loop through the cluster elements searching for new layer numbers'''
                # grab the first cell_geo_ID, l-th element of the j-th cluster
                _cellGeoTag = _cluster_cell_ID[j][l]

                # find the index of the cell_geo_ID, use [0][0] becuase of np.where return
                _cellGeo_idx = np.argwhere(np.asarray(_cellGeoID == _cellGeoTag))

                # find the sampling layer based on the index
                _this_layer = _cellGeoSampling[_cellGeo_idx]

                if not np.isin(_this_layer, _event_layers):
                    _event_layers.append(_this_layer)      

        # add event to list if one is in the barrel layer
        if np.any(np.isin(_event_layers, _filter)):
            _barrel_events.append(_evt_idx)

    return np.array(_barrel_events)


def center_cluster():
    ''' Expects a single cluster with a single cluster center '''
    return None

def center_clusters():
    ''' Expects multiple clusters and creates an imshow using the average of the cluster centers'''
    return None

def cluster_to_imshow():
    ''' Takes a cluster and converts to a python imshow '''
    return None

In [1]:
cPionEvents = 'atlas_images/pipm/'
nPionEvents = 'atlas_images/pi0/'

In [6]:
cPion_502 = ur.open(cPionEvents+'user.angerami.24559744.OutputStream._000502.root')
nPion_502 = ur.open(nPionEvents+'user.angerami.24559740.OutputStream._000502.root')

In [7]:
cPionEvents = cPion_502["EventTree"]
nPionEvents = nPion_502["EventTree"]

In [8]:
cellGeoTree = cPion_502["CellGeo"]

Load&Show Tree

In [9]:
# Show keys
cPionEvents.show(name_width=42,
                interpretation_width=60)

## Event information
## Charged Pions
ak_cluster_nCells = cPionEvents.arrays(filter_name="cluster_nCells")
ak_cluster_cell_ID = cPionEvents.arrays(filter_name="cluster_cell_ID")
ak_cluster_cell_E = cPionEvents.arrays(filter_name="cluster_cell_E")
nCluster = cPionEvents.arrays(library='np')["nCluster"]
eventNumber = cPionEvents.arrays(library='np')["eventNumber"]
nTrack = cPionEvents.arrays(library='np')["nTrack"]
cPionPart = cPionEvents.arrays(library='np')["nTruthPart"]
cPionId = cPionEvents.arrays(library='np')["truthPartPdgId"]
ak_cluster_Eta = cPionEvents.arrays(filter_name="cluster_Eta")
ak_cluster_Phi = cPionEvents.arrays(filter_name="cluster_Phi")

name                                       | typename                 | interpretation                                              
-------------------------------------------+--------------------------+-------------------------------------------------------------
runNumber                                  | int32_t                  | AsDtype('>i4')
eventNumber                                | int64_t                  | AsDtype('>i8')
lumiBlock                                  | int32_t                  | AsDtype('>i4')
coreFlags                                  | uint32_t                 | AsDtype('>u4')
mcEventNumber                              | int32_t                  | AsDtype('>i4')
mcChannelNumber                            | int32_t                  | AsDtype('>i4')
mcEventWeight                              | float                    | AsDtype('>f4')
nTruthPart                                 | int32_t                  | AsDtype('>i4')
G4PreCalo_n_EM                        